# Backtest

In [1]:
from system.agent import Agent
from system.env import Env

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


config = {"capital" : 100, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2023", "end" : "2023"
          }

env = Env(**config)

agentIds = [1, 2, 3, 4]
env.config_agents(agentIds)

params_1 = {"agentId" : 1, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name" : "MOMENTUM"}
params_2 = {"agentId" : 2, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA"}
params_3 = {"agentId" : 3, "symbol" : "ETH", "allocation" : 100, "env" : env, "policy_name":"MOMENTUM"}
params_4 = {"agentId" : 4, "symbol" : "ETH", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA"}

In [2]:
agent_1 = Agent(**params_1)
agent_1.update_policy_params(params = 3)
agent_1.run_episode()

agent_2 = Agent(**params_2)
agent_2.update_policy_params(params = (7, 14, 21))
agent_2.run_episode()

agent_3 = Agent(**params_3)
agent_3.update_policy_params(params = 5)
agent_3.run_episode()

agent_4 = Agent(**params_4)
agent_4.update_policy_params(params = (7, 14, 21))
agent_4.run_episode()

In [3]:
env.get_report(agentId=1, symbol = "BTC")

## Global Report

In [5]:
env.trades[4]

,agentId,price,quantity,position,side,state,in_value,out_value,value,pnl,pnl_pct,symbol,status,gp,cum_gp,rets,ret_price,price_cum,loss,recovery
date,,,,,,,,,,,,,,,,,,,,
2023-02-14,4,1555.71,0.064279,1,LONG,"('Open', 'LONG', False, False)",100.000000,0.000000,100.000000,0.000000,0.000000,ETH,Open,0.000000,0.000000,0.000000,NaN,NaN,0.0,0.0
2023-02-15,4,1674.92,0.064279,1,LONG,"('-', 'LONG', False, False)",100.000000,0.000000,107.662739,7.662739,0.076627,ETH,-,7.662739,7.662739,0.076627,0.076627,1.076627,0.0,0.0
2023-02-16,4,1637.84,0.064279,1,LONG,"('-', 'LONG', False, False)",100.000000,0.000000,105.279262,5.279262,0.052793,ETH,-,-2.383478,5.279262,-0.023835,-0.022138,1.052793,0.0,0.0
2023-02-17,4,1693.87,0.064279,1,LONG,"('-', 'LONG', False, False)",100.000000,0.000000,108.880833,8.880833,0.088808,ETH,-,3.601571,8.880833,0.036016,0.034210,1.088808,0.0,0.0
2023-02-18,4,1695.34,0.000000,0,None,"('Close', 'LONG', False, False)",0.000000,108.975323,0.000000,8.975323,0.089753,ETH,Close,0.094491,8.975323,0.000945,0.000868,1.089753,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-20,4,1603.89,0.087215,1,LONG,"('-', 'LONG', False, False)",135.614408,0.000000,139.883592,4.269184,0.031480,ETH,-,3.254871,39.883592,0.024001,0.023823,1.030970,0.0,0.0
2023-10-21,4,1628.94,0.000000,0,None,"('Close', 'LONG', False, False)",0.000000,142.068333,0.000000,6.453925,0.047590,ETH,Close,2.184741,42.068333,0.016110,0.015618,1.047072,0.0,0.0
2023-10-22,4,1663.70,0.000000,0,None,"('-', None, False, False)",0.000000,0.000000,0.000000,0.000000,0.000000,ETH,-,0.000000,42.068333,0.000000,0.021339,1.069415,0.0,0.0


In [ ]:
env.tradesData

,agentId,price,quantity,position,side,state,in_value,out_value,value,pnl,pnl_pct,symbol,status
date,,,,,,,,,,,,,
2023-02-14,1,22199.84,0.004505,1,LONG,"('Open', 'LONG', False, False)",100.000000,0.000000,100.000000,0.000000,0.000000,BTC,NaN
2023-02-15,1,24324.05,0.004505,1,LONG,"('-', 'LONG', False, False)",100.000000,0.000000,109.568582,9.568582,0.095686,BTC,NaN
2023-02-16,1,23517.72,0.004505,1,LONG,"('-', 'LONG', False, False)",100.000000,0.000000,105.936439,5.936439,0.059364,BTC,NaN
2023-02-17,1,24569.97,0.004505,1,LONG,"('-', 'LONG', False, False)",100.000000,0.000000,110.676338,10.676338,0.106763,BTC,NaN
2023-02-18,1,24579.42,0.004505,1,LONG,"('-', 'LONG', False, False)",100.000000,0.000000,110.718906,10.718906,0.107189,BTC,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-20,4,1603.89,0.087215,1,LONG,"('-', 'LONG', False, False)",135.614408,0.000000,139.883592,4.269184,0.031480,ETH,NaN
2023-10-21,4,1628.94,0.000000,0,None,"('Close', 'LONG', False, False)",0.000000,142.068333,0.000000,6.453925,0.047590,ETH,NaN
2023-10-22,4,1663.70,0.000000,0,None,"('-', None, False, False)",0.000000,0.000000,0.000000,0.000000,0.000000,ETH,NaN


In [16]:
env.globalReport()

#  MultiAgent System

In [ ]:
from magent import MAgentThread
from master import  MasterAgentThread
from system.env import Env

from threading import Thread, Condition, Event, Barrier, Semaphore

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


config = {"capital" : 100, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2023", "end" : "2023"
          }
env = Env(**config)


def custom_action():
    print(" *** Barrier passee *** ")


condition = Condition()
barrier = Barrier(2, action=custom_action)

agentId_1 = 1
agentId_2 = 2

agent_msg = {agentId_1 : None, agentId_2 : None}
master_msg = {agentId_1 : None, agentId_2 : None}

activeAgent = 1

boss_params = {"condition" : condition, "activeAgent" : activeAgent ,  "agent_msg" : agent_msg, "master_msg" : master_msg}

params_1 = {"agentId" : agentId_1, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name" : "MOMENTUM",
            "condition" : condition, "barrier" : barrier, "agent_msg" : agent_msg, "master_msg" : master_msg}

params_2 = {"agentId" : agentId_2, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA",
            "condition" : condition, "barrier" : barrier, "agent_msg" : agent_msg, "master_msg" : master_msg}


Boss = MasterAgentThread(**boss_params)
Boss.addAgent(1)
Boss.addAgent(2)

agent_1 = MAgentThread(**params_1)
agent_1.update_policy_params(params = 3)

agent_2 = MAgentThread(**params_2)
agent_2.update_policy_params(params = (7, 14, 21))

In [ ]:
Boss.start()
agent_1.start()
agent_2.start()


agent_1.join()
agent_2.join()
Boss.join()

In [ ]:
trades = env.journal.trades_data.copy()
trades.head()

In [ ]:
dd = env.journal.trades_data
dd.head

In [ ]:
import plotly.figure_factory as ff

dd1 = dd[dd["agentId"] == 1]
dd2 = dd[dd["agentId"] == 2]


# Group data together
hist_data = [dd1["pnl"], dd2["pnl"]]

group_labels = ['Agent 1', 'Agent 2']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2)
fig.show()

In [ ]:
len(dd2)

In [ ]:
len(dd1)

In [ ]:
from plotly.figure_factory import create_distplot
import numpy as np
import pandas as pd

df = pd.DataFrame({'2012': np.random.randn(200), 
                   '2013': np.random.randn(200)+1}
                  )
fig = create_distplot([df[c] for c in df.columns], df.columns)
fig.show()

In [ ]:
import plotly.figure_factory as ff
import numpy as np
np.random.seed(1)

x = np.random.randn(1000)
hist_data = [x]
group_labels = ['distplot'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels)
fig.show()

In [ ]:
import plotly.figure_factory as ff
import numpy as np

# Add histogram data
x1 = np.random.randn(200) - 2
x2 = np.random.randn(200)
x3 = np.random.randn(200) + 2
x4 = np.random.randn(200) + 4

# Group data together
hist_data = [x1, x2, x3, x4]

group_labels = ['Group 1', 'Group 2', 'Group 3', 'Group 4']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2)
fig.show()

In [ ]:
import plotly.figure_factory as ff
import numpy as np

x1 = np.random.randn(200)
x2 = np.random.randn(200) + 2

group_labels = ['Group 1', 'Group 2']

colors = ['slategray', 'magenta']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot([x1, x2], group_labels, bin_size=.5,
                         curve_type='normal', # override default 'kde'
                         colors=colors)

# Add title
fig.update_layout(title_text='Distplot with Normal Distribution')
fig.show()

In [ ]:
import plotly.figure_factory as ff
import numpy as np

x1 = np.random.randn(200) - 1
x2 = np.random.randn(200)
x3 = np.random.randn(200) + 1

hist_data = [x1, x2, x3]

group_labels = ['Group 1', 'Group 2', 'Group 3']
colors = ['#333F44', '#37AA9C', '#94F3E4']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors)

# Add title
fig.update_layout(title_text='Curve and Rug Plot')
fig.show()

In [ ]:
import plotly.graph_objects as go

categories = ['processing cost','mechanical properties','chemical stability',
              'thermal stability', 'device integration']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[1, 5, 2, 2, 3],
      theta=categories,
      fill='toself',
      name='Product A'
))
fig.add_trace(go.Scatterpolar(
      r=[4, 3, 2.5, 1, 2],
      theta=categories,
      fill='toself',
      name='Product B'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=False
)

fig.show()